In [1]:
from datasets import load_dataset

# 加载数据集
dataset = load_dataset("EricWiener/llm4html-descgen")

# 查看数据集的字段
print(dataset["train"].column_names)


D:\conda_envs\finalproject\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['snippet', 'description']


In [13]:
from transformers import T5Tokenizer

# 初始化 T5 分词器
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_function(examples):
    # 将 HTML 内容与分类任务标签拼接
    inputs = [f"Classify: {snippet}" for snippet in examples["snippet"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # 将类别标签转化为模型的目标输出，并启用 padding 和 truncation
    labels = tokenizer(examples["description"], max_length=10, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 处理整个数据集
tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [15]:
tokenized_dataset 

DatasetDict({
    train: Dataset({
        features: ['snippet', 'description', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 66346
    })
    validation: Dataset({
        features: ['snippet', 'description', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 9800
    })
    test: Dataset({
        features: ['snippet', 'description', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8844
    })
})

In [9]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

from transformers import DataCollatorWithPadding

import torch

# 检查 GPU 可用性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained("t5-small")
model.to(device)


training_args = TrainingArguments(
    output_dir="D:/model_output",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',          
    logging_steps=1000,             
)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator  # data_collator
)

# finetuning
trainer.train()


C:\Users\Xuewen Sun\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.381200,0.347570
2,0.319700,0.319878
3,0.275000,0.314831


TrainOutput(global_step=49761, training_loss=0.3733001290112048, metrics={'train_runtime': 6412.0684, 'train_samples_per_second': 31.041, 'train_steps_per_second': 7.761, 'total_flos': 2.6938161481383936e+16, 'train_loss': 0.3733001290112048, 'epoch': 3.0})

In [3]:
import torch
print("PyTorch CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)


PyTorch CUDA available: True
CUDA version: 12.1


In [23]:
import torch
import os
from sklearn.metrics import accuracy_score

# 启用 expandable_segments
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 设置设备为 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# classifier function
def classify_with_t5(snippet):
    input_text = f"Classify: {snippet}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=256, truncation=True).input_ids.to(device)
    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=50, num_beams=2)
    classification = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return classification

# 清除显存缓存
torch.cuda.empty_cache()

# 执行逐条推理
true_labels = dataset["test"]["description"]
predicted_labels = []
for snippet in dataset["test"]["snippet"]:
    predicted_labels.append(classify_with_t5(snippet))
    torch.cuda.empty_cache()  

# 计算准确率
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"模型准确率: {accuracy * 100:.2f}%")



模型准确率: 65.16%


In [21]:
import torch
torch.cuda.empty_cache()


In [25]:
# 保存模型权重
model.save_pretrained("D:/model_output/final_model")
# 保存分词器
tokenizer.save_pretrained("D:/model_output/final_model")

('D:/model_output/final_model\\tokenizer_config.json',
 'D:/model_output/final_model\\special_tokens_map.json',
 'D:/model_output/final_model\\spiece.model',
 'D:/model_output/final_model\\added_tokens.json')